# Data Transformation
------

Transform the data into a format so that we can insert it into the time series transformer<br>

Model input dimensions:<br>
Axis 0 (nested): ID, sample<br>
Axis 1 : Variable
<br>

Goal of the transformation: <br>
Axis 0: Participant ID, time point<br>
Axis 1: Channel

In [1]:
import cedalion
import cedalion.nirs
import cedalion.xrutils as xrutils
from cedalion.datasets import get_multisubject_fingertapping_snirf_paths
import numpy as np
import xarray as xr
import pint
import matplotlib.pyplot as p
import scipy.signal
import os.path

import pandas as pd

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score

xr.set_options(display_max_rows=3, display_values_threshold=50)
np.set_printoptions(precision=4)
try:
    cedalion.units.define('@alias meter = o')
except:
    pass

In [2]:
fnames = get_multisubject_fingertapping_snirf_paths()
subjects  = [f"sub-{i:02d}" for i in [1,2,3,4,5]]

# store data of different subjects in a dictionary
data = {} 
for subject,fname in zip(subjects, fnames):
    print(fname)
    elements = cedalion.io.read_snirf(fname)

    amp = elements[0].data[0]
    stim = elements[0].stim # pandas Dataframe
    geo3d = elements[0].geo3d
    
    # cedalion registers an accessor (attribute .cd ) on pandas DataFrames
    stim.cd.rename_events( {
        "1.0" : "control",
        "2.0" : "Tapping/Left",
        "3.0" : "Tapping/Right"
    })
    
    dpf = xr.DataArray([6, 6], dims="wavelength", coords={"wavelength" : amp.wavelength})
    
    data[subject] = xr.Dataset(
        data_vars = {
            "amp" : amp,
            "od"  : - np.log( amp / amp.mean("time") ),
            "geo" : geo3d,
            "conc": cedalion.nirs.beer_lambert(amp, geo3d, dpf)
        },
        attrs={"stim" : stim}, # store stimulus data in attrs
        coords={"subject" : subject} # add the subject label as a coordinate
    )

/Users/idalabbackup/Library/Caches/cedalion/multisubject-fingertapping.zip.unzip/BIDS-NIRS-Tapping/sub-01/nirs/sub-01_task-tapping_nirs.snirf
/Users/idalabbackup/Library/Caches/cedalion/multisubject-fingertapping.zip.unzip/BIDS-NIRS-Tapping/sub-02/nirs/sub-02_task-tapping_nirs.snirf
/Users/idalabbackup/Library/Caches/cedalion/multisubject-fingertapping.zip.unzip/BIDS-NIRS-Tapping/sub-03/nirs/sub-03_task-tapping_nirs.snirf
/Users/idalabbackup/Library/Caches/cedalion/multisubject-fingertapping.zip.unzip/BIDS-NIRS-Tapping/sub-04/nirs/sub-04_task-tapping_nirs.snirf
/Users/idalabbackup/Library/Caches/cedalion/multisubject-fingertapping.zip.unzip/BIDS-NIRS-Tapping/sub-05/nirs/sub-05_task-tapping_nirs.snirf


In [3]:
display(data["sub-01"])

Magnitude,[[[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882] [0.1856806 0.186377 0.1836514 ... 0.1856486 0.1850836 0.1842172]] [[0.227516 0.2297024 0.2261366 ... 0.2264519 0.2271665 0.226713] [0.6354927 0.637668 0.6298023 ... 0.6072068 0.6087293 0.6091066]] [[0.1064704 0.1066212 0.1053444 ... 0.121114 0.1205022 0.1205441] [0.2755033 0.2761615 0.2727006 ... 0.2911952 0.2900544 0.2909847]] ... [[0.2027881 0.1996586 0.2004866 ... 0.2318743 0.2311941 0.2330808] [0.4666358 0.4554404 0.4561614 ... 0.4809749 0.4812827 0.4862896]] [[0.4885007 0.4802285 0.4818338 ... 0.6109142 0.6108118 0.613845] [0.8457658 0.825988 0.8259648 ... 0.975894 0.9756599 0.9826459]] [[0.6304559 0.6284427 0.6287045 ... 0.6810626 0.6809573 0.6818709] [1.2285622 1.2205907 1.2190002 ... 1.2729124 1.2727222 1.2755645]]]
Units,dimensionless
Magnitude,[[[0.040420720851589244 0.04460046098449573 0.0442158667266921 ... 0.010876348689028297 0.01189058853204466 0.006847636450666034] [0.023820496835156947 0.020076986081598715 0.03480909472899116 ... 0.023992850639162802 0.027040875264083337 0.031732993935654014]] [[-0.008280063468686781 -0.01784405593180633 -0.002198739340328898 ... -0.003592058220542012 -0.006742726672860488 -0.0047443982578777594] [-0.03725579234490354 -0.04067296052603699 -0.028261149412724014 ... 0.008275386235838757 0.005771141488662178 0.0051515177576299375]] [[0.1005582275517943 0.09914287354277777 0.11119025710690507 ... -0.02830701101541003 -0.023242770227372977 -0.023590421283804962] [0.04993799639529682 0.047551763623795935 0.06016311306879892 ... -0.005456229532249399 -0.0015308884573803712 -0.004733085331725723]] ... [[0.09543410489184105 0.11098678842280141 0.10684828479073329 ... -0.038599718102985896 -0.03566192105523218 -0.04378947818067423] [0.03858010875753767 0.06286432773409747 0.06128249638988582 ... 0.008314104856099373 0.007674359302522161 -0.0026751405572218796]] [[0.15506580435158798 0.17214467649471782 0.16880746748060918 ... -0.06854981498765861 -0.06838218329243445 -0.07333574448986935] [0.10250044669095155 0.12616269025036977 0.1261907782199553 ... -0.040611038136814874 -0.040371126750177955 -0.04750589562552556]] [[0.0580532188103781 0.06125157285758024 0.06083507429037254 ... -0.019157798285875216 -0.019003174994086804 -0.020343916456372766] [0.02437701777814723 0.030886638430615728 0.03219054575814673 ... -0.01108594305570406 -0.010936510768482734 -0.013167265399560904]]]
Units,dimensionless
Magnitude,[[-0.041613204679326624 0.026799775287857947 0.1299043936308115] [-0.06476686499872276 0.05814256998996063 0.0908425773727145] [-0.07120554551675068 -0.012874272652217859 0.10787860947691345] [-0.0859043654400404 0.018971698468891116 0.06509762433137256] [0.03694171596700852 0.02748380530252158 0.13022129709104263] [0.06065133742692848 0.05882414589197514 0.09117717995727878] [0.06712771392323756 -0.012199231886346213 0.1085725493643022] [0.08188685574250908 0.020427932162352107 0.06571325110115192] [-0.037619588707178915 0.06322851630256272 0.11572802770110814] [-0.04134445059646741 -0.011779611291995052 0.13495002938154654] [-0.07242424650162711 0.02347293206381116 0.10322218957482163] [-0.07912592748234686 0.05140929117919257 0.057370046083468226] [0.03352717285472944 0.06359968341212022 0.11583881331702946] [0.03686639505686032 -0.011397164907962862 0.13536724076864515] [0.06791592703520163 0.02468254467119271 0.10366605207860985] [0.075310088095807 0.05226884499005337 0.05787698428594235] [-0.03773895423262196 0.034082658086024245 0.1294919790818403] [-0.061454307897075164 0.06443800208211416 0.09061004226260877] [-0.07282878975853647 -0.00527870527992114 0.10743054838539287] [-0.08439610638498087 0.02706123378098264 0.06559510739262155] [0.040013338219712126 0.020439745814301982 0.13063767506528579] [0.06428020193514211 0.05162125732852231 0.09133632943784001] [0.06521393141744246 -0.019260368037897515 0.10880928230870081] [0.08272091030272573 0.012990608473329186 0.06658402323335233] [-0.082489991830

In [4]:
data["sub-01"].time.samples[-1] / data["sub-01"].time.time[-1]

<xarray.DataArray ()> Size: 8B
array(7.8125)
Coordinates:
    time     float64 8B 2.974e+03
    samples  int64 8B 23238
    subject  <U6 24B 'sub-01'

In [5]:
con = data["sub-01"].conc
con

Magnitude,[[[0.13358239209978304 0.003830283928282647 0.3485663033081772 ... 0.44265804481800175 0.5026875842869235 0.6632956216585129] [-0.7839218209250456 -0.7640062849016895 -0.6372321468890523 ... 0.23076741243710222 0.20512673319586014 0.16981311715887412] [0.11212379127914074 0.0731324774375613 0.23388667595717802 ... 0.1596030122051564 0.19659631898706498 0.12887590142000407] ... [-0.08996858531937893 0.29760277238751065 0.3035425193755599 ... 0.5709685540656639 0.5273568764191445 0.37714069302428915] [0.9042366397918427 1.3400782049340045 1.3813304247520295 ... -0.27197257740205305 -0.2674864774443464 -0.4012836512581511] [-0.18579734906426823 0.4272187017064305 0.6069404693046493 ... -0.3055763013235321 -0.2958772890477925 -0.4927704425869438]] [[0.4302890462699156 0.5291122886883187 0.39408017947069485 ... -0.03812950019946504 -0.048780953537845895 -0.16954954368367178] [0.19718416808530376 0.07475108502790183 0.21473067649191507 ... -0.13048136047941516 -0.15862865728380518 -0.12125825687699171] [1.1069744947570999 1.1055519748193772 1.1824229590870727 ... -0.38395393707435815 -0.34006765489373475 -0.31841423445257] ... [1.1310069656643422 1.1631091585413527 1.1132916692449675 ... -0.6597509742064699 -0.6094788888477872 -0.6460548286777947] [1.8250852563797992 1.8988537984724563 1.8366005072863651 ... -0.855162631152548 -0.8545173888400667 -0.8731183537487607] [3.6718816282381956 3.6383236051532477 3.5444730339882793 ... -1.0728951396816504 -1.0669728023777476 -1.0756417750550116]]]
Units,micromolar


In [6]:
def pad_along_axis(array: np.ndarray, target_length: int, axis: int = 0) -> np.ndarray:

    pad_size = target_length - array.shape[axis]

    if pad_size <= 0:
        return array

    npad = [(0, 0)] * array.ndim
    npad[axis] = (0, pad_size)

    return np.pad(array, pad_width=npad, mode='constant', constant_values=0)

In [7]:
from typing import Dict

def into_dataframe(subject_array:Dict[str, xr.DataArray]) -> pd.DataFrame:
    """Turns a dictionary with subject data in xarray form into a TST compatible dataframe.
     
     WARNING: Currently, channels are simply concatenated.
        This means that the channel sorting depends on the ordering of channels within the data.
        Therefore it is prespecified by the data that is read.
    """
    
    # Find out longest sample length
    longest_ts = 0
    for sub, d in subject_array.items():
        if d.conc.shape[-1] > longest_ts:
            longest_ts = d.conc.shape[-1]
    print(f"Padding data to longest sequence with length: {longest_ts}")

    # Transform matrix into TST shape
    all_subs = []
    for sub in subject_array.values():
        b = pad_along_axis(np.array(sub.conc), longest_ts, axis=-1)
        c = np.concatenate(b, axis=0)
        all_subs.append(c)

    padded = np.stack(all_subs, 0)
    padded_2d = np.concatenate(padded, -1).T

    # create index
    n_subjects = np.arange(padded.shape[0])
    n_samples = longest_ts

    # Creates an index for each subject
    index = np.repeat(n_subjects, n_samples)

    df = pd.DataFrame(data=padded_2d, index=index)

    return df

test = into_dataframe(data)
test.head()

Padding data to longest sequence with length: 23239


/usr/local/Caskroom/miniconda/base/envs/cedalion/lib/python3.10/site-packages/xarray/core/variable.py:306: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  data = np.asarray(data)


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,0.133582,-0.783922,0.112124,-6.605575,0.072065,0.157804,-0.165179,-2.991802,-0.662061,-0.650387,...,1.657113,1.165608,1.689860,1.595643,0.979688,1.081886,-1.353489,1.131007,1.825085,3.671882
0,0.003830,-0.764006,0.073132,-6.785065,0.105426,0.146239,-0.140950,-2.917413,-0.457271,-0.544232,...,1.710501,1.213622,1.719345,1.582520,0.993582,1.190909,-1.235771,1.163109,1.898854,3.638324
0,0.348566,-0.637232,0.233887,-6.464715,0.297150,0.405250,0.073547,-2.674065,-0.352645,-0.511028,...,1.707331,1.272164,1.695244,1.649482,0.950616,1.073023,-1.287167,1.113292,1.836601,3.544473
0,0.191469,-0.696672,0.328634,-6.319006,0.257867,0.375040,0.020626,-2.615859,-0.316476,-0.322001,...,1.758502,1.176429,1.697077,1.736220,1.089578,1.207084,-1.291210,1.138696,1.885575,3.566036
0,0.415568,-0.553111,0.426234,-6.267730,0.263309,0.466229,0.016447,-2.560593,-0.215540,-0.329690,...,1.679600,1.200704,1.705122,1.539560,1.074913,1.232196,-1.363738,1.045494,1.869924,3.492447


# READ IN CUSTOM STUDY DATA
-------
Given any generic directory, find all snirf files and combine them based on subject

In [8]:
import pathlib

vfc = pathlib.Path('../fnirs_data/vfc_high_density/')
fnames = sorted(list(vfc.rglob("*.snirf")))
subjects  = [f"sub-{i:02d}" for i in range(1, len(fnames)+1)]
# store data of different subjects in a dictionary
data = {} 
for subject,fname in zip(subjects, fnames):
    print(fname)
    elements = cedalion.io.read_snirf(fname)

    amp = elements[0].data[0]
    stim = elements[0].stim # pandas Dataframe
    geo3d = elements[0].geo3d
    # cedalion registers an accessor (attribute .cd ) on pandas DataFrames
    # stim.cd.rename_events( {
    #     "1.0" : "control",
    #     "2.0" : "Tapping/Left",
    #     "3.0" : "Tapping/Right"
    # })
    # TODO: Talk to Eike whether this is needed
    
    dpf = xr.DataArray([6, 6], dims="wavelength", coords={"wavelength" : amp.wavelength})
    
    data[subject] = xr.Dataset(
        data_vars = {
            "amp" : amp,
            "od"  : - np.log( amp / amp.mean("time") ),
            "geo" : geo3d,
            "conc": cedalion.nirs.beer_lambert(amp, geo3d, dpf)
        },
        attrs={"stim" : stim}, # store stimulus data in attrs
        coords={"subject" : subject} # add the subject label as a coordinate
    )


../fnirs_data/vfc_high_density/sub-01/ses-02/nirs/nirs.snirf
../fnirs_data/vfc_high_density/sub-06/ses-01/nirs/sub-06_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-08/ses-01/nirs/sub-08_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-09/ses-01/nirs/sub-09_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-11/ses-01/nirs/sub-11_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-12/ses-01/nirs/sub-12_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-13/ses-01/nirs/sub-13_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-14/ses-01/nirs/sub-14_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-15/ses-01/nirs/sub-15_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-17/ses-01/nirs/sub-17_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-20/ses-01/nirs/sub-20_

/usr/local/Caskroom/miniconda/base/envs/cedalion/lib/python3.10/site-packages/pint/facets/numpy/numpy_func.py:307: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)
/usr/local/Caskroom/miniconda/base/envs/cedalion/lib/python3.10/site-packages/pint/facets/numpy/numpy_func.py:307: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)


../fnirs_data/vfc_high_density/sub-24/ses-01/nirs/sub-24_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-25/ses-01/nirs/sub-25_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-26/ses-01/nirs/sub-26_ses-01_task-WordStroop_run-01_nirs.snirf
../fnirs_data/vfc_high_density/sub-27/ses-01/nirs/sub-27_ses-01_task-WordStroop_run-01_nirs.snirf


/usr/local/Caskroom/miniconda/base/envs/cedalion/lib/python3.10/site-packages/pint/facets/numpy/numpy_func.py:307: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)
/usr/local/Caskroom/miniconda/base/envs/cedalion/lib/python3.10/site-packages/pint/facets/numpy/numpy_func.py:307: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)


In [9]:
len(data.keys())

17

In [10]:
test = into_dataframe(data)
print(test.shape)
test.head()

Padding data to longest sequence with length: 11535


/usr/local/Caskroom/miniconda/base/envs/cedalion/lib/python3.10/site-packages/xarray/core/variable.py:306: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  data = np.asarray(data)


(196095, 428)


,0,1,2,3,4,5,6,7,8,9,...,418,419,420,421,422,423,424,425,426,427
0,-0.453743,-0.564715,-0.377385,-0.824645,-0.841382,-0.610995,0.435185,0.553305,-0.612950,-0.658194,...,1.110651,0.032406,0.360596,0.408561,0.706638,0.673319,0.427281,0.941846,0.862790,0.045816
0,-0.528239,-0.617457,-0.428352,-0.877047,-1.042706,-0.718649,-0.005403,0.516707,-0.666865,-0.581255,...,1.444965,0.052339,0.369538,0.360814,0.697427,0.694215,0.293682,0.927723,0.830082,-0.007813
0,-0.556833,-0.653591,-0.441188,-0.924813,-1.139435,-0.800622,-0.020688,0.508694,-0.742322,-0.557370,...,0.664919,-0.016350,0.350973,-0.354544,0.666938,0.663754,0.331351,0.949278,0.801049,-0.255246
0,-0.457469,-0.622616,-0.402936,-0.872273,-0.955246,-0.670531,0.707054,0.498206,-0.592676,-0.463784,...,0.431476,0.050230,0.378439,1.584142,0.697774,0.662882,0.223749,0.992944,0.809655,0.005065
0,-0.427606,-0.542865,-0.339193,-0.797756,-0.873110,-0.564392,0.985954,0.508709,-0.487986,-0.672742,...,0.580905,0.068477,0.385932,1.106890,0.683338,0.672764,0.307312,0.999986,0.838859,0.122165


In [15]:
TRAIN = test[:round((len(test) * 4/5))]
TRAIN.shape

(156876, 428)

In [16]:
TEST = test[round((len(test) * 4/5)):]
TEST.shape

(39219, 428)

In [13]:
TRAIN.to_csv("../fnirs_data/csv_format/vfc_high_density_TRAIN.csv")
TEST.to_csv("../fnirs_data/csv_format/vfc_high_density_TEST.csv")


mini_train = TRAIN[TRAIN.index < 3]
mini_test = TEST[TEST.index < 2]

mini_train.to_csv("/Users/idalabbackup/code/mvts_transformer_/imputation/vfc_high_density_TRAIN_mini.csv")
mini_test.to_csv("/Users/idalabbackup/code/mvts_transformer_/imputation/vfc_high_density_TEST_mini.csv")



# OPEN QUESTIONS
-----
- How to determine location based on xarray?
- If we remove noisy channels and fill with 0, can we also pad with 0?
- Should we just pad to a constant for our experiments?
- Mention raw file in load snirf
- Do we need to include this into cedalion fast? (dependency issues very likely)
- Talk to Eike about attributes?
- How to cut data for testing?
- Look into transduction?
- Add a learnable convolutional layer
- Does feature exclusion also work for testing data? 

In [14]:
print(f"{list(range(428//2, 428))}")

[214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413,